<a href="https://colab.research.google.com/github/danemiles/projects/blob/main/FF_Draft_Prep_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Editable Variables & Setup**

---



a. Projections

In [245]:
# LEAGUE VARIABLES
teams = 12
starting_qb = 1
starting_rb = 2
starting_wr = 2
starting_te = 1
starting_flex = 2
bench = 7

# scoring
rush_yd = 0.1
rush_td = 6
fumb = -2

ppr = 1
rec_yd = 0.1
rec_td = 6

p_yd = 0.04
p_td = 6
pick = -2

ret_td = 6


######
# calc starters by pos
lst_qb = teams*starting_qb
lst_rb = teams*starting_rb
lst_wr = teams*starting_wr
lst_te = teams*starting_te
lst_flex = teams*starting_flex

In [246]:
! pip install sportsipy

In [247]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import lxml.html as lh


In [248]:
url='https://betiq.teamrankings.com/fantasy-football/rankings/yahoo/?size=10'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [249]:
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')

# Obtain information from tag <table>
table1 = soup.find('table', id='fantasy-table')

# Obtain every title of columns with tag <th>
headers = []
for i in table1.find_all('th'):
 title = i.text
 headers.append(title)

headers
#rank appears to be the first true header


['Projections last updated: Aug  1, 2023',
 'Passing',
 'Rushing',
 'Receiving',
 'Fantasy',
 'Rank',
 'Player',
 'Pos',
 'Team',
 'Bye',
 'Yds',
 'TD',
 'Int',
 'Yds',
 'TD',
 'Rec',
 'Yds',
 'TD',
 'FPts',
 'Value']

In [250]:
#drop header values before 'Rank'
del headers[:5]
headers

['Rank',
 'Player',
 'Pos',
 'Team',
 'Bye',
 'Yds',
 'TD',
 'Int',
 'Yds',
 'TD',
 'Rec',
 'Yds',
 'TD',
 'FPts',
 'Value']

In [251]:
#make the dataframe
mydata = pd.DataFrame(columns = headers)

mydata

,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds,TD,Rec,Yds,TD,FPts,Value


In [252]:
# Create a for loop to fill mydata
for j in table1.find_all('tr')[2:]:
 row_data = j.find_all('td')
 row = [i.text for i in row_data]
 length = len(mydata)
 mydata.loc[length] = row

In [253]:
#check accuracy
mydata

,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds,TD,Rec,Yds,TD,FPts,Value
0,1,Christian McCaffrey,RB1,SF,9,0,0.0,0.0,919,7.4,71,566,3.1,246.9,156.3
1,2,Austin Ekeler,RB2,LAC,5,0,0.0,0.0,786,6.4,74,586,4.3,238.3,147.7
2,3,Justin Jefferson,WR1,MIN,13,0,0.0,0.0,17,0.4,104,1434,8.9,252.7,139.3
3,4,Bijan Robinson,RB3,ATL,11,0,0.0,0.0,1146,8.4,38,331,1.8,227.9,137.3
4,5,Ja'Marr Chase,WR2,CIN,7,0,0.0,0.0,14,0.1,101,1347,9.9,246.5,133.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,Greg Dortch,WR89,ARI,14,0,0.0,0.0,26,0.0,42,402,1.9,75.3,-38.1
296,295,Joshua Kelley,RB78,LAC,5,0,0.0,0.0,283,2.1,8,58,0.3,52.5,-38.1
297,298,J.J. Taylor,RB79,NE,11,0,0.0,0.0,260,1.7,12,83,0.3,52.2,-38.4
298,299,Russell Gage,WR91,TB,5,0,0.0,0.0,28,0.1,37,385,2.4,74.7,-38.7


In [254]:
#fix duplicate header names.
#source: https://stackoverflow.com/questions/40774787/renaming-columns-in-a-pandas-dataframe-with-duplicate-column-names

class renamer():
  def __init__(self):
    self.d = dict()

  def __call__(self, x):
    if x not in self.d:
      self.d[x] = 0
      return x
    else:
      self.d[x] += 1
      return "%s_%d" % (x, self.d[x])

mydata = mydata.rename(columns=renamer())
mydata

#here, sections are PASSING, RUSHING, RECEIVING

,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds_1,TD_1,Rec,Yds_2,TD_2,FPts,Value
0,1,Christian McCaffrey,RB1,SF,9,0,0.0,0.0,919,7.4,71,566,3.1,246.9,156.3
1,2,Austin Ekeler,RB2,LAC,5,0,0.0,0.0,786,6.4,74,586,4.3,238.3,147.7
2,3,Justin Jefferson,WR1,MIN,13,0,0.0,0.0,17,0.4,104,1434,8.9,252.7,139.3
3,4,Bijan Robinson,RB3,ATL,11,0,0.0,0.0,1146,8.4,38,331,1.8,227.9,137.3
4,5,Ja'Marr Chase,WR2,CIN,7,0,0.0,0.0,14,0.1,101,1347,9.9,246.5,133.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,Greg Dortch,WR89,ARI,14,0,0.0,0.0,26,0.0,42,402,1.9,75.3,-38.1
296,295,Joshua Kelley,RB78,LAC,5,0,0.0,0.0,283,2.1,8,58,0.3,52.5,-38.1
297,298,J.J. Taylor,RB79,NE,11,0,0.0,0.0,260,1.7,12,83,0.3,52.2,-38.4
298,299,Russell Gage,WR91,TB,5,0,0.0,0.0,28,0.1,37,385,2.4,74.7,-38.7


In [255]:
# mydata.dtypes #check data types ... likely has char/object values
test = mydata
test = test.replace('--',0)

test['Bye'] = test['Bye'].astype(int, errors='raise')

In [256]:
conv_to_float = ['Yds','TD','Int','Yds_1','TD_1','Rec','Yds_2','TD_2','FPts','Value'] # cols to be transformed to float

# run loop to change strings to float
for i in conv_to_float:
  test[i] = test[i].astype(float,errors='raise')

test.info() # check for completion

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rank    300 non-null    object 
 1   Player  300 non-null    object 
 2   Pos     300 non-null    object 
 3   Team    300 non-null    object 
 4   Bye     300 non-null    int64  
 5   Yds     300 non-null    float64
 6   TD      300 non-null    float64
 7   Int     300 non-null    float64
 8   Yds_1   300 non-null    float64
 9   TD_1    300 non-null    float64
 10  Rec     300 non-null    float64
 11  Yds_2   300 non-null    float64
 12  TD_2    300 non-null    float64
 13  FPts    300 non-null    float64
 14  Value   300 non-null    float64
dtypes: float64(10), int64(1), object(4)
memory usage: 37.5+ KB


In [257]:
df = test
df

,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds_1,TD_1,Rec,Yds_2,TD_2,FPts,Value
0,1,Christian McCaffrey,RB1,SF,9,0.0,0.0,0.0,919.0,7.4,71.0,566.0,3.1,246.9,156.3
1,2,Austin Ekeler,RB2,LAC,5,0.0,0.0,0.0,786.0,6.4,74.0,586.0,4.3,238.3,147.7
2,3,Justin Jefferson,WR1,MIN,13,0.0,0.0,0.0,17.0,0.4,104.0,1434.0,8.9,252.7,139.3
3,4,Bijan Robinson,RB3,ATL,11,0.0,0.0,0.0,1146.0,8.4,38.0,331.0,1.8,227.9,137.3
4,5,Ja'Marr Chase,WR2,CIN,7,0.0,0.0,0.0,14.0,0.1,101.0,1347.0,9.9,246.5,133.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,Greg Dortch,WR89,ARI,14,0.0,0.0,0.0,26.0,0.0,42.0,402.0,1.9,75.3,-38.1
296,295,Joshua Kelley,RB78,LAC,5,0.0,0.0,0.0,283.0,2.1,8.0,58.0,0.3,52.5,-38.1
297,298,J.J. Taylor,RB79,NE,11,0.0,0.0,0.0,260.0,1.7,12.0,83.0,0.3,52.2,-38.4
298,299,Russell Gage,WR91,TB,5,0.0,0.0,0.0,28.0,0.1,37.0,385.0,2.4,74.7,-38.7


In [258]:
# create new projected points based on previously entered league settings

df["ProjPts"] = (
  (df["Yds"]*p_yd) +
  (df["TD"]*p_td) +
  (df["Int"]*pick) +
  (df["Yds_1"]*rush_yd) +
  (df["TD_1"]*rush_td) +
  (df["Rec"]*ppr) +
  (df["Yds_2"]*rec_yd) +
  (df["TD_2"]*rec_td)
)

# create new position / rank field
df['Pos_old'] = df['Pos']
df['Pos'] = df.Pos.str[:2] # take left two chars

df

,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds_1,TD_1,Rec,Yds_2,TD_2,FPts,Value,ProjPts,Pos_old
0,1,Christian McCaffrey,RB,SF,9,0.0,0.0,0.0,919.0,7.4,71.0,566.0,3.1,246.9,156.3,282.5,RB1
1,2,Austin Ekeler,RB,LAC,5,0.0,0.0,0.0,786.0,6.4,74.0,586.0,4.3,238.3,147.7,275.4,RB2
2,3,Justin Jefferson,WR,MIN,13,0.0,0.0,0.0,17.0,0.4,104.0,1434.0,8.9,252.7,139.3,304.9,WR1
3,4,Bijan Robinson,RB,ATL,11,0.0,0.0,0.0,1146.0,8.4,38.0,331.0,1.8,227.9,137.3,246.9,RB3
4,5,Ja'Marr Chase,WR,CIN,7,0.0,0.0,0.0,14.0,0.1,101.0,1347.0,9.9,246.5,133.1,297.1,WR2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,Greg Dortch,WR,ARI,14,0.0,0.0,0.0,26.0,0.0,42.0,402.0,1.9,75.3,-38.1,96.2,WR89
296,295,Joshua Kelley,RB,LAC,5,0.0,0.0,0.0,283.0,2.1,8.0,58.0,0.3,52.5,-38.1,56.5,RB78
297,298,J.J. Taylor,RB,NE,11,0.0,0.0,0.0,260.0,1.7,12.0,83.0,0.3,52.2,-38.4,58.3,RB79
298,299,Russell Gage,WR,TB,5,0.0,0.0,0.0,28.0,0.1,37.0,385.0,2.4,74.7,-38.7,93.3,WR91


In [259]:
# make positional ranking column

df['Pos_rnk'] = df.groupby('Pos')['ProjPts'].rank(ascending=False)
df['Pos_rnk'] = df['Pos_rnk'].astype(int,errors='raise') # convert to int
df['Rnk'] = df['Pos'] + df['Pos_rnk'].astype(str)
#df['Rank'] = df['ProjPts'].rank(ascending=False)

df.head(12)


,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds_1,TD_1,Rec,Yds_2,TD_2,FPts,Value,ProjPts,Pos_old,Pos_rnk,Rnk
0,1,Christian McCaffrey,RB,SF,9,0.0,0.0,0.0,919.0,7.4,71.0,566.0,3.1,246.9,156.3,282.5,RB1,1,RB1
1,2,Austin Ekeler,RB,LAC,5,0.0,0.0,0.0,786.0,6.4,74.0,586.0,4.3,238.3,147.7,275.4,RB2,2,RB2
2,3,Justin Jefferson,WR,MIN,13,0.0,0.0,0.0,17.0,0.4,104.0,1434.0,8.9,252.7,139.3,304.9,WR1,1,WR1
3,4,Bijan Robinson,RB,ATL,11,0.0,0.0,0.0,1146.0,8.4,38.0,331.0,1.8,227.9,137.3,246.9,RB3,3,RB3
4,5,Ja'Marr Chase,WR,CIN,7,0.0,0.0,0.0,14.0,0.1,101.0,1347.0,9.9,246.5,133.1,297.1,WR2,3,WR3
5,6,Tyreek Hill,WR,MIA,10,0.0,0.0,0.0,33.0,0.6,102.0,1344.0,8.9,245.4,132.0,296.7,WR3,4,WR4
6,7,Cooper Kupp,WR,LAR,10,0.0,0.0,0.0,41.0,0.4,108.0,1285.0,9.2,244.3,130.9,298.2,WR4,2,WR2
7,7,Nick Chubb,RB,CLE,5,0.0,0.0,0.0,1232.0,9.5,26.0,222.0,1.0,221.5,130.9,234.4,RB4,6,RB6
8,9,Tony Pollard,RB,DAL,7,0.0,0.0,0.0,983.0,8.1,44.0,358.0,2.0,216.7,126.1,238.7,RB5,4,RB4
9,10,Derrick Henry,RB,TEN,7,0.0,0.0,0.0,1249.0,9.5,22.0,186.0,0.5,214.4,123.8,225.5,RB6,9,RB9


b. Avg Draft Position (ADP)

---



In [260]:
# source: https://www.fantasypros.com/nfl/adp/ppr-overall.php
url='https://www.fantasypros.com/nfl/adp/ppr-overall.php'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [261]:
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')

# Obtain information from tag <table>
table1 = soup.find('table', id='data')

# Obtain every title of columns with tag <th>
headers = []
for i in table1.find_all('th'):
 title = i.text
 headers.append(title)

headers



['Rank',
 'Player Team (Bye)',
 'POS',
 'ESPN',
 'Sleeper',
 'NFL',
 'RTSports',
 'FFC',
 'AVG']

In [262]:
# make the dataframe
adp = pd.DataFrame(columns = headers)

adp

,Rank,Player Team (Bye),POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG


In [263]:
# Create a for loop to fill adp
for j in table1.find_all('tr')[1:]:
 row_data = j.find_all('td')
 row = [i.text for i in row_data]
 length = len(adp)
 adp.loc[length] = row

adp.head()

,Rank,Player Team (Bye),POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG
0,1,Justin Jefferson MIN (13),WR1,1,1,1,1,,1.0
1,2,Christian McCaffrey SF (9),RB1,3,2,2,2,,2.3
2,3,Austin Ekeler LAC (5),RB2,2,4,3,4,,3.3
3,4,Ja'Marr Chase CIN (7),WR2,4,3,4,3,,3.5
4,5,Cooper Kupp LAR (10),WR3,5,6,5,5,,5.3


In [264]:
# need to split player name / team name / bye into separate cols

# where POS ne "DST"
#   Find "(", go left some amount?  = playername
#   etc
# where POS eq "DST", Find "(", trim to there

In [265]:
# for sake of trying, parse table in Pandas, not with BS4
url = 'https://www.fantasypros.com/nfl/adp/ppr-overall.php'
tables = pd.read_html(url) # Returns list of all tables on page
adp_2 = tables[0] # Select table of interest
adp_2

# significantly cleaner and easier... will remember moving fwd

,Rank,Player Team (Bye),POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG
0,1,Justin Jefferson MIN (13),WR1,1.0,1.0,1.0,1.0,NaN,1.0
1,2,Christian McCaffrey SF (9),RB1,3.0,2.0,2.0,2.0,NaN,2.3
2,3,Austin Ekeler LAC (5),RB2,2.0,4.0,3.0,4.0,NaN,3.3
3,4,Ja'Marr Chase CIN (7),WR2,4.0,3.0,4.0,3.0,NaN,3.5
4,5,Cooper Kupp LAR (10),WR3,5.0,6.0,5.0,5.0,NaN,5.3
...,...,...,...,...,...,...,...,...,...
545,546,Davis Mills HOU (7),QB79,496.0,NaN,NaN,NaN,NaN,496.0
546,547,Teddy Bridgewater,QB80,497.0,NaN,NaN,NaN,NaN,497.0
547,548,Skylar Thompson MIA (10),QB81,498.0,NaN,NaN,NaN,NaN,498.0
548,549,Matthew Slater NE (11),WR180,499.0,NaN,NaN,NaN,NaN,499.0


**2. Creating VORP, VBD**

---


In [266]:
# how many starters in league by position (total). Dub xhat_pos
# take proj tot pts for each player, proj_pts - xhat_pos = VORP

In [267]:
df.head()

,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds_1,TD_1,Rec,Yds_2,TD_2,FPts,Value,ProjPts,Pos_old,Pos_rnk,Rnk
0,1,Christian McCaffrey,RB,SF,9,0.0,0.0,0.0,919.0,7.4,71.0,566.0,3.1,246.9,156.3,282.5,RB1,1,RB1
1,2,Austin Ekeler,RB,LAC,5,0.0,0.0,0.0,786.0,6.4,74.0,586.0,4.3,238.3,147.7,275.4,RB2,2,RB2
2,3,Justin Jefferson,WR,MIN,13,0.0,0.0,0.0,17.0,0.4,104.0,1434.0,8.9,252.7,139.3,304.9,WR1,1,WR1
3,4,Bijan Robinson,RB,ATL,11,0.0,0.0,0.0,1146.0,8.4,38.0,331.0,1.8,227.9,137.3,246.9,RB3,3,RB3
4,5,Ja'Marr Chase,WR,CIN,7,0.0,0.0,0.0,14.0,0.1,101.0,1347.0,9.9,246.5,133.1,297.1,WR2,3,WR3


In [268]:
qbs = df.loc[df['Pos'] == 'QB']
rbs = df.loc[df['Pos'] == 'RB']
wrs = df.loc[df['Pos'] == 'WR']
tes = df.loc[df['Pos'] == 'TE']

rp_qb = round(qbs.ProjPts.nlargest(teams*starting_qb).iloc[-1],1)
rp_rb = round(rbs.ProjPts.nlargest(teams*starting_rb).iloc[-1],1)
rp_wr = round(wrs.ProjPts.nlargest(teams*starting_wr).iloc[-1],1)
rp_te = round(tes.ProjPts.nlargest(teams*starting_te).iloc[-1],1)

print(rp_qb, rp_rb, rp_wr, rp_te)

328.4 158.2 198.0 130.2


In [269]:
df['RP'] = np.where(df['Pos']=='RB', rp_rb,
                   np.where(df['Pos']=='QB', rp_qb,
                   np.where(df['Pos']=='WR', rp_wr, rp_te)))

df['VORP'] = df['ProjPts'] - df['RP']

df.head()

,Rank,Player,Pos,Team,Bye,Yds,TD,Int,Yds_1,TD_1,...,Yds_2,TD_2,FPts,Value,ProjPts,Pos_old,Pos_rnk,Rnk,RP,VORP
0,1,Christian McCaffrey,RB,SF,9,0.0,0.0,0.0,919.0,7.4,...,566.0,3.1,246.9,156.3,282.5,RB1,1,RB1,158.2,124.3
1,2,Austin Ekeler,RB,LAC,5,0.0,0.0,0.0,786.0,6.4,...,586.0,4.3,238.3,147.7,275.4,RB2,2,RB2,158.2,117.2
2,3,Justin Jefferson,WR,MIN,13,0.0,0.0,0.0,17.0,0.4,...,1434.0,8.9,252.7,139.3,304.9,WR1,1,WR1,198.0,106.9
3,4,Bijan Robinson,RB,ATL,11,0.0,0.0,0.0,1146.0,8.4,...,331.0,1.8,227.9,137.3,246.9,RB3,3,RB3,158.2,88.7
4,5,Ja'Marr Chase,WR,CIN,7,0.0,0.0,0.0,14.0,0.1,...,1347.0,9.9,246.5,133.1,297.1,WR2,3,WR3,198.0,99.1


In [270]:
# remove unneeded cols
df = df.drop(columns=['Yds', 'TD','Int','Yds_1','TD_1','Rec','Yds_2','TD_2','FPts','Value'])

# let's also remove defense and kickers
values = ['DS','K1','K2','K3','K4','K5','K6','K7','K8','K9']

#drop rows that contain any value in the list
df = df[df.Pos.isin(values) == False]

In [271]:
# add VORP rankings, total and by Pos
df['VORP_rnk_pos'] = df.groupby('Pos')['VORP'].rank(ascending=False).astype(int,errors='raise')

df['VORP_rnk_tot'] = df['VORP'].rank(ascending=False).astype(int,errors='raise')
df.head()

,Rank,Player,Pos,Team,Bye,ProjPts,Pos_old,Pos_rnk,Rnk,RP,VORP,VORP_rnk_pos,VORP_rnk_tot
0,1,Christian McCaffrey,RB,SF,9,282.5,RB1,1,RB1,158.2,124.3,1,2
1,2,Austin Ekeler,RB,LAC,5,275.4,RB2,2,RB2,158.2,117.2,2,3
2,3,Justin Jefferson,WR,MIN,13,304.9,WR1,1,WR1,198.0,106.9,1,5
3,4,Bijan Robinson,RB,ATL,11,246.9,RB3,3,RB3,158.2,88.7,3,11
4,5,Ja'Marr Chase,WR,CIN,7,297.1,WR2,3,WR3,198.0,99.1,3,8


In [275]:
# Time for analysis. Let's see who the top players are by total VORP
df.sort_values(by='VORP_rnk_tot', ascending=True).head(10)

,Rank,Player,Pos,Team,Bye,ProjPts,Pos_old,Pos_rnk,Rnk,RP,VORP,VORP_rnk_pos,VORP_rnk_tot
11,12,Travis Kelce,TE,KC,10,275.60,TE1,1,TE1,130.2,145.40,1,1
0,1,Christian McCaffrey,RB,SF,9,282.50,RB1,1,RB1,158.2,124.30,1,2
1,2,Austin Ekeler,RB,LAC,5,275.40,RB2,2,RB2,158.2,117.20,2,3
16,17,Josh Allen,QB,BUF,13,437.32,QB1,1,QB1,328.4,108.92,1,4
2,3,Justin Jefferson,WR,MIN,13,304.90,WR1,1,WR1,198.0,106.90,1,5
24,25,Patrick Mahomes,QB,KC,10,433.34,QB3,2,QB2,328.4,104.94,2,6
6,7,Cooper Kupp,WR,LAR,10,298.20,WR4,2,WR2,198.0,100.20,2,7
4,5,Ja'Marr Chase,WR,CIN,7,297.10,WR2,3,WR3,198.0,99.10,3,8
5,6,Tyreek Hill,WR,MIA,10,296.70,WR3,4,WR4,198.0,98.70,4,9
25,26,Joe Burrow,QB,CIN,7,424.34,QB4,3,QB3,328.4,95.94,3,10




---


# **Trying Something New:**

###_Machine Learning on Historical FF Data_

#### _by Dane Miles, beginning 4 Aug 2023_

In [287]:
# import pandas as pd
datayr = 2021

data = pd.read_csv(f'https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/{datayr}.csv')
data.head()

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,...,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,...,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,...,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,...,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,...,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,...,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


In [307]:
# add points based on scoring standards abocve
data["FantasyPts"] = (
  (data["PassingYds"]*p_yd) +
  (data["PassingTD"]*p_td) +
  (data["Int"]*pick) +
  (data["RushingYds"]*rush_yd) +
  (data["RushingTD"]*rush_td) +
  (data["Rec"]*ppr) +
  (data["ReceivingYds"]*rec_yd) +
  (data["ReceivingTD"]*rec_td)
)

data = data.fillna(0) # remove NaN

In [308]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [309]:
# consider running mutliple models inc/excl Team Name, Pos, etc.
X = data.drop(['Player', 'Pos','FantasyPts'], axis=1)
X = pd.get_dummies(data=X, drop_first=True)
y = data['FantasyPts']

# Step 4: Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



In [310]:
# Step 5: Model Training
model = LinearRegression()
model.fit(X_train, y_train)

# Step 6: Model Evaluation on Validation Set
y_val_pred = model.predict(X_val)
mse_val = mean_squared_error(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)

print("Validation MSE:", mse_val)
print("Validation MAE:", mae_val)



Validation MSE: 3.9606276076123143e-26
Validation MAE: 1.0400934101005268e-13


In [311]:
# Step 7: Hyperparameter Tuning (Optional)
# You can perform hyperparameter tuning here, depending on the chosen model.

# Step 8: Model Validation on Test Set
y_test_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print("Test MSE:", mse_test)
print("Test MAE:", mae_test)

# Step 9: Model Deployment (Optional)
# Deploy the model to make predictions for new players.

# Step 10: Continuous Improvement
# Regularly update the model with new data to improve its accuracy over time.

Test MSE: 2.6488794878315004e-26
Test MAE: 8.770705113623146e-14




---

We have a basic setup established here.
Next steps:

1.   Build graphs
2.   Evalute model choice (ours is a placeholder)
3.   Consider add'l data -- ADP? Games injured? Coaching systems? (i.e. would have to have coach / year combos... how would we handle midseason changes?)

